# 1. Preparation

## 1.1 Install pip and torch

In [1]:
import lmod
await lmod.purge(force=True)
await lmod.load('compiler/gnu/13.3')

In [2]:
import sys
# sys.executable 应该指向你的虚拟环境中的 Python 解释器。
# sys.path 中应该包含你虚拟环境的 site-packages 目录。
print(sys.executable)
print(sys.path)

/pfs/data5/home/kit/stud/unyfv/myEnv/bin/python
['/pfs/data5/home/kit/stud/unyfv', '', '/home/kit/stud/unyfv/.local/lib/python3.9/site-packages', '/opt/bwhpc/common/jupyter/tensorflow/2023-10-10/lib/python3.9/site-packages', '/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '/pfs/data5/home/kit/stud/unyfv/myEnv/lib64/python3.9/site-packages', '/pfs/data5/home/kit/stud/unyfv/myEnv/lib/python3.9/site-packages']


In [3]:
import os
# 更新 PYTHONPATH 和 PATH
os.environ["PYTHONPATH"] = "/pfs/data5/home/kit/stud/unyfv/myEnv/lib/python3.9/site-packages:" + os.environ.get("PYTHONPATH", "")
os.environ["PATH"] = "/pfs/data5/home/kit/stud/unyfv/myEnv/bin:" + os.environ["PATH"]
# 验证更新
!which python
!which pip
!echo $PYTHONPATH

/pfs/data5/home/kit/stud/unyfv/myEnv/bin/python
/pfs/data5/home/kit/stud/unyfv/myEnv/bin/pip
/pfs/data5/home/kit/stud/unyfv/myEnv/lib/python3.9/site-packages:


In [4]:
!pip install pip==24.0
!pip show torch | grep Version

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Version: 2.5.1


## 1.2 Install fairseq

First install fairseq, trust me, the original installation method sucks...

In [5]:
!git clone https://github.com/facebookresearch/fairseq.git
%cd fairseq
!pip install --editable ./ 

fatal: destination path 'fairseq' already exists and is not an empty directory.
/pfs/data5/home/kit/stud/unyfv/fairseq
Obtaining file:///pfs/data5/home/kit/stud/unyfv/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp39-cp39-linux_x86_64.whl size=9573 sha256=c64f4be617890d0fec28b1628d602597e4a6ebe303d24ae737c0c873124bebe8
  Stored in directory: /scratch/slurm_tmpdir/job_24586128/pip-ephem-wheel-cache-bh_kh7nb/wheels/04/f3/80/b3eeb7f36a7a1c71a666614a9545aa3a57c1e9d004ff1e7236
Successfully built fairseq
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or co

Then we need to add a new environment variable so that we can use the fairseq command in the terminal.  

In [6]:
!echo $PYTHONPATH
os.environ['PYTHONPATH'] += ":/pfs/data5/home/kit/stud/unyfv/fairseq/"
!echo $PYTHONPATH

/pfs/data5/home/kit/stud/unyfv/myEnv/lib/python3.9/site-packages:
/pfs/data5/home/kit/stud/unyfv/myEnv/lib/python3.9/site-packages::/pfs/data5/home/kit/stud/unyfv/fairseq/


## 1.3 Install other packages

In [7]:
!pip install sacremoses
!pip install sentencepiece
!pip install sacrebleu

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
DEPRECATION: omegaconf 2.0.6 has a non-sta

## 1.4 Activate GPU

In [8]:
import torch
print(torch.__version__)
if torch.cuda.is_available():
    device = torch.cuda.current_device(); print('Current device: ', torch.cuda.get_device_name(device))
else:
    device = 'cpu'; print('Current device: CPU.')

2.1.0+cu121
Current device:  Tesla V100-SXM2-32GB


So far, all packages have been installed.
From now on, just execute the following cells.

# 2. Data Preparation

## 2.1 Download dataset

Here we still use TED-dataset as example.

In [9]:
!wget -nc -O sample_data.zip https://bwsyncandshare.kit.edu/s/Xx3D56SJmG8PwXj/download
!unzip sample_data.zip -d dataset

--2024-11-01 14:58:27--  https://bwsyncandshare.kit.edu/s/Xx3D56SJmG8PwXj/download
Resolving bwsyncandshare.kit.edu (bwsyncandshare.kit.edu)... 2a00:1398:b::8d03:8781, 141.3.135.129
Connecting to bwsyncandshare.kit.edu (bwsyncandshare.kit.edu)|2a00:1398:b::8d03:8781|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘sample_data.zip’

sample_data.zip         [     <=>            ]  28.19M  23.2MB/s    in 1.2s    

2024-11-01 14:58:29 (23.2 MB/s) - ‘sample_data.zip’ saved [29555676]

Archive:  sample_data.zip
   creating: dataset/sample_data/
 extracting: dataset/sample_data/dev.de-en.de  
 extracting: dataset/sample_data/dev.de-en.en  
 extracting: dataset/sample_data/train.de-en.de  
 extracting: dataset/sample_data/train.de-en.en  
 extracting: dataset/sample_data/tst.de-en.de  
 extracting: dataset/sample_data/tst.de-en.en  


## 2.2 Preprocessing

Segment the text into subwords using BPE.

In [11]:
import sentencepiece as spm

# After execution, you can find two bpe files in the directory.
spm.SentencePieceTrainer.train(input="dataset/sample_data/train.de-en.en,dataset/sample_data/train.de-en.de",
                               model_prefix="bpe",
                               vocab_size=10000)

print('Finished training sentencepiece model.')

Finished training sentencepiece model.


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: dataset/sample_data/train.de-en.en
  input: dataset/sample_data/train.de-en.de
  input_format: 
  model_prefix: bpe
  model_type: UNIGRAM
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pa

Then we use the trained segmentation model to preprocess the sentences from train/dev/test sets:

In [12]:
# Load the trained sentencepiece model
spm_model = spm.SentencePieceProcessor(model_file="bpe.model")

# Important: encoding="utf-8"
for partition in ["train", "dev", "tst"]:
    for lang in ["de", "en"]:
        f_out = open(f"dataset/sample_data/spm.{partition}.de-en.{lang}", "w", encoding="utf-8")

        with open(f"dataset/sample_data/{partition}.de-en.{lang}", "r", encoding="utf-8") as f_in:
            for line_idx, line in enumerate(f_in.readlines()):
                # Segmented into subwords
                line_segmented = spm_model.encode(line.strip(), out_type=str)
                # Join the subwords into a string
                line_segmented = " ".join(line_segmented)
                f_out.write(line_segmented + "\n")

        f_out.close()

Now, we will binarize the data for training with fairseq.  
Feel free to check the [documentation](https://fairseq.readthedocs.io/en/latest/command_line_tools.html) of fairseq commands.

In [13]:
# Preprocess/binarize the data
TEXT="dataset/sample_data"
!echo $TEXT
# Binarize the data for training
!fairseq-preprocess \
    --source-lang en --target-lang de \
    --trainpref $TEXT/spm.train.de-en \
    --validpref $TEXT/spm.dev.de-en \
    --testpref $TEXT/spm.tst.de-en \
    --destdir binarized_data/iwslt14.de-en \
    --thresholdtgt 0 --thresholdsrc 0 \
    --workers 8

dataset/sample_data
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq_cli.preprocess:Namespace(no_progress_bar=False, log_interval=100, log_format=None, log_file=None, aim_repo=None, aim_run_hash=None, tensorboard_logdir=None, wandb_project=None, azureml_logging=False, seed=1, cpu=False, tpu=False, bf16=False, memory_efficient_bf16=False, fp16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, on_cpu_convert_precision=False, min_loss_scale=0.0001, threshold_loss_scale=None, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, quantization_config_path=None, profile=False, reset_logging=False, suppress_crashes=False, use_plasma_view=False, plasma_path='/tmp/plasma', criterion='cross_entropy', tokenizer=None, bpe=None, optimizer=None, lr_scheduler='

The data preprocessing is completed.

# 3. Training

In [14]:
# Make sure that (0.9, 0.98) in "", error might occurs when use ''.
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    binarized_data/iwslt14.de-en \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas "(0.9, 0.98)" --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --keep-last-epochs 2 \
    --max-tokens 4096 \
    --max-epoch 20 \
    --fp16

2024-11-01 15:02:14 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2024-11-01 15:02:17 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': Fals

# 4. Decoding

Now we can generate translations with the trained model.

In [15]:
# ↓↓↓ Even delete this line, it works...xD
# TEST_INPUT="dataset/spm.tst.de-en.de"

!fairseq-generate binarized_data/iwslt14.de-en \
      --task translation \
      --source-lang en \
      --target-lang de \
      --path checkpoints/checkpoint_best.pt \
      --batch-size 256 \
      --beam 4 \
      --remove-bpe=sentencepiece > "en-de.decode.log"

INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
DEBUG:hydra.core.utils:Setting JobRuntime:name=UNKNOWN_NAME
DEBUG:hydra.core.utils:Setting JobRuntime:name=utils
INFO:fairseq_cli.generate:{'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantiz

We extract the hypotheses and references from the decoding log file.

In [18]:
%%bash
grep ^H "en-de.decode.log" | sed 's/^H-//g' | cut -f 3 | sed 's/ ##//g' > ./hyp.txt
grep ^T "en-de.decode.log" | sed 's/^T-//g' | cut -f 2 | sed 's/ ##//g' > ./ref.txt
head ./hyp.txt
echo ""
head ./ref.txt

Hier ist eine andere Gruppe.
Es geht um den Tod.
Und ich begann zu fallen.
Sie werden es wahrscheinlich nicht.
Ich kenne Amira.
Stattdessen ist meine Herausforderung das.
Es war nur eine Geschichte.
Könnte das wahr sein?
Dana: So.
Wo sind die verschiedenen Länder?

Hier ist eine andere Gruppe.
Es geht um Tod.
Ich begann zu fallen.
Sie tun es wahrscheinlich nicht.
Ich kenne Amira.
Meine Herausforderung besteht eher darin:
Es war nur eine Geschichte.
Kann das wirklich möglich sein?
Dana: Da lang.
Ich kann Ihnen Afrika zeigen.


# Section 5. Evaluation

Here we use BLEU as example.

In [17]:
!bash -c "cat hyp.txt | sacrebleu ref.txt"

{
 "name": "BLEU",
 "score": 27.1,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "60.5/33.9/21.2/13.6 (BP = 0.977 ratio = 0.977 hyp_len = 52243 ref_len = 53465)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}
